[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [2]:
import requests

## Список объектов

In [2]:
# Для скачивания данных по

offset_ = 1
limit_ = 100
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
objects_data.get('data').get('list')[0]

{'objId': 13383,
 'developer': {'devId': 306,
  'shortName': '2МЕН ГРУПП',
  'fullName': '2МЕН ГРУПП',
  'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
  'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
  'companyGroup': 5776,
  'devInn': '7701651356'},
 'rpdRegionCd': 72,
 'objAddr': 'г Тюмень',
 'objFloorMin': 4,
 'objFloorMax': 20,
 'objElemLivingCnt': 68,
 'objReady100PercDt': '2022-12-31',
 'objSquareLiving': 6293.78,
 'objCompensFundFlg': 1,
 'objProblemFlg': 0,
 'latitude': 57.1519,
 'longitude': 65.5536,
 'problemFlag': 'NONE',
 'siteStatus': 'Строящиеся',
 'objGuarantyEscrowFlg': 0,
 'residentialBuildings': 1,
 'rpdNum': '72-000095',
 'pdId': 28105,
 'buildType': 'Жилое',
 'objStatus': 0}

In [3]:
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
objids[:10]

[13383, 13397, 13391, 13398, 13399, 36912, 44076, 23494, 27476, 10891]

In [81]:
len(objids)

100

# Данные по определенному объекту

In [86]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/10891'

In [87]:
res = requests.get(url)
res.json()

{'data': {'id': 10891,
  'pdId': 23054,
  'developer': {'devId': 3006,
   'devShortCleanNm': 'ААК ВОЛОГДААГРОСТРОЙ',
   'devShortNm': 'ААК ВОЛОГДААГРОСТРОЙ',
   'devFullCleanNm': 'АКЦИОНЕРНАЯ АГРОСТРОИТЕЛЬНАЯ КОМПАНИЯВОЛОГДААГРОСТРОЙ',
   'problObjCnt': 0,
   'buildObjCnt': 1,
   'comissObjCnt': 5,
   'regRegionDesc': 'Волгоградская область',
   'devPhoneNum': '+7(817)274-79-00',
   'devSite': 'vologdaagrostroy.ru',
   'devEmail': 'vologdaagrostroy@yandex.ru',
   'devInn': '3525000888',
   'devOgrn': '1023500886386',
   'devKpp': '352501001',
   'devLegalAddr': 'Волгоградская область, г Вологда, улица Галкинская дом 24',
   'devFactAddr': 'обл Вологодская, г Вологда, улица Галкинская, д. 24',
   'lastRpdId': 625041,
   'fundGuarantyFlg': 1,
   'devOrgRegRegionCd': 34,
   'devEmplMainFullNm': 'Корытин Владимир Николаевич',
   'developerGroupName': 'ВологдаАгрострой',
   'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
   'companyGroupId': 7368,
   'objGuarant

Домашнее задание

In [24]:
import pandas as pd

In [101]:
dfs = pd.DataFrame()
for ids in objids:
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' + str(ids)
    res = requests.get(url)
    dfs=dfs.append(pd.json_normalize(res.json().get('data')))
dfs.to_excel('stop_loc.xlsx')
dfs.to_pickle("./dummy.pkl")  

In [ ]:
dfs.to_sql('base', con=connection)